# Win32com and Automating Outlook  
@ftlphys  
Michael Gilbert  
3-23-16

### What does Win32com and pythoncom allow you to do?
*  Hook into microsoft com system  (win32ole / win32api)
    - control/access outlook
    - control/access word & excel
    - control/access internet explorer
    - control/access some system services
    - ...
* Eliminate corporate "email hell" through automation (can even bypass certain annoyances in the group policy ;-) )!

### Getting everything Installed...
* Installed from sourceforge website of the [Python for Windows Extension](https://sourceforge.net/projects/pywin32/)
    -  PyPi does not have the latest version, so download the appropriate exe from sourceforge
    -  If using Anaconda, it does have the the latest, use conda install pywin32 (build 220 as of this presentation)

### First a word about the documentation...
* It is sparse...
* [Active State Docs](http://docs.activestate.com/activepython/2.4/pywin32/html/com/win32com/HTML/QuickStartClientCom.html)
    - it's old and does not cover all options, but has some simple examples and explains the general structure of the library
* [Microsoft Outlook Iterop](https://msdn.microsoft.com/en-us/library/microsoft.office.interop.outlook.mailitem_properties.aspx)
    - covers everything you can do with outlook
    - written with examples in VBA and/or C#, so will require translation
* Stack overflow is probably the best place to find information about using win32com

### Some considerations when interacting with Outlook...
* win32com.client gives you access to hooking into outlook
* pythoncom manages the communications (not always a required import), especially important in multithreaded/multiprocessed access to outlook
    -  Outlook does not like multiple commands from multiple hooks occuring simultaneously and will freeze
* pywintypes is needed to handle exceptions

### Let's see some code and control Outlook!!

In [1]:
# Required imports to interact with outlook
import win32com.client
import pythoncom
import pywintypes

### Using Dispatch to send an email...

In [ ]:
try:
    # hook into outlook (may very depending upon your companies implementation of login credentials)
    o = win32com.client.Dispatch("Outlook.Application")

    # create a new email message
    Msg = o.CreateItem(0)

    # fill in the necessary fields
    Msg.To = 'joe@gmail.com'
    Msg.Subject = "Let's have lunch!"
    Msg.Body = 'How does Saturday at noon sound?'

    # send it!
    Msg.Send()
except pywintypes.com_error as e:
    # handle errors with the com ineraction
    print(e)

### Using Dispatch if you have multiple hooks in outlook (multi-processed, multi-threaded, or multi-programs)...

In [ ]:
import os
try:
    # Initialize the pythoncom to support multiple threads/processes
    pythoncom.CoInitialize()
    # hook into outlook (may vary depending upon your companies implementation of login credentials)
    o = win32com.client.Dispatch("Outlook.Application")

    # create a new email message
    Msg = o.CreateItem(0)

    # fill in the necessary fields
    Msg.To = 'joe@gmail.com'
    Msg.Subject = "Funny picture"
    # let's send an html email with an embedded image attached
    attachment1 = 'attachment.png'
    attaching = Msg.Attachments.Add(attachment1, 1, 1, 'image_label')
    
    # imageCid and PropertyAccessor are only necessary if your outlook email has
    # restrictions on ways of embedding images (particularly for view in certain apps)
    # You can get the necessary properties by reading the html source of emails 
    # you send normally through outlook...
    imageCid = os.path.basename('attachment_path') + '@123'
    attaching.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001E", imageCid)
    Msg.HTMLBody = '<html><body> ' + 'Funny picture' + ":<br /> <img src=\"cid:{0}\"></body></html>".format(imageCid)

    # send it!
    Msg.Send()
    # close the coinitialization to clean up threads
    pythoncom.CoUninitialize()
except pywintypes.com_error as e:
    # handle errors with the com ineraction
    print(e)

### Accessing various folders...

In [ ]:
o = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
# The folder name may vary for your inbox, different numbers will correspond to different folders
inbox = o.GetDefaultFolder(6)
# Get all messages in the Inbox
messages = reversed(inbox.Items)
for i, f in enumerate(messages):
    print(i, f.Subject)
    if i == 10:
        break

### How about monitoring and/or responding to all incoming emails from a specific individual?
* Use Dispath with Events
* Create a pythoncom eventloop

In [ ]:
# imported sys only to deal with flushing print statement due to juypter notebook's issues with event loops
import sys

class EmailMonitor:
    outlook_hook = None

    # method name is recognized by dispatch with events for new incoming mail
    def OnNewMailEx(self, receivedItemsIDs):
        for ID in receivedItemsIDs.split(","):
            mail = EmailMonitor.outlook_hook.Session.GetItemFromID(ID)
            subject = mail.Subject
            sender = mail.SenderName
            print('Recieved an Email!')
            print(sender, subject)
            sys.stdout.flush()

    # method name is recognized by dispatch with events for sent mail
    def OnItemSend(self, item, cancel):
        print('Sent an Email!')
        print(item.Subject)
        sys.stdout.flush()


def main():
    # below 3 lines are only needed for compatability with pyinstaller or the
    # genchache will not work when compiled to an executable with the lastest
    # version of pyinstaller
    sys.modules['win32com.gen_py.os'] = None
    sys.modules['win32com.gen_py.pywintypes'] = None
    sys.modules['win32com.gen_py.pythoncom'] = None
    # hook into outlook to capture all events for example sent/recieved emails
    outlook_hook = win32com.client.DispatchWithEvents("Outlook.Application", EmailMonitor)
    # optionally setting class attribute link to hook
    EmailMonitor.outlook_hook=outlook_hook
    # create pythoncom event loop
    pythoncom.PumpMessages()
    
    
main()

Sent an Email!
Hey!
Recieved an Email!
(u'Charles Gilbert', u'Hey!')


### Looking for more ways to automate tasks and reduce redundancy?
* [*Automate the Boring stuff with Python*](https://www.nostarch.com/automatestuff)
    - GUI/form automation
    - great regex tutorial
    - accessing/creating excel documents automatically using openpyxl
    - much more!!